# Pertemuan 11
- Feature Detection & Feature Description
    - Harris Corner Detection
    - Shi-Tomasi Corner Detector & Good Features to Track
    - Introduction to SIFT (Scale-Invariant Feature Transform)
    - Introduction to SURF (Speeded-Up Robust Features)
    - FAST Algorithm for Corner Detection
- Feature Matching
    - Brute-Force based matcher
    - FLANN based matcher
___
### Maximizing Jetson Nano Perfomance

In [ ]:
# sudo nvpmodel -m 0
# sudo jetson_clocks

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# check OpenCV Version

cv2.__version__

___
# 1. Feature Detection 
- **Feature Detection** is process to find specific unique area on image like a corner or edge.
- With this definition we understand that feature have a **large variation in intensity in all the directions** on image.<br><br>
- At the below of image, six small image patches are given. 
- Question for you is to find the exact location of these patches in the original image. 
- How many correct results can you find?<br><br>
    ![](resources/feature_building.jpg)<br><br>
- A and B are flat surfaces and they are **spread over a lot of area**. 
    - It is difficult to find the exact location of these patches.
- C and D are much more simple. 
    - They are **edges** of the building. 
    - You can find an approximate location, but exact location is still difficult. 
- E and F are some corners of the building. 
    - And they can be easily found. Because at the **corners**, wherever you move this patch, it will look different.
<br><br><br>
- So now we move into simpler (and widely used image) for better understanding.<br>
    ![](resources/feature_simple.png)<br><br>
- the **blue patch** is **flat area** and difficult to find and track. 
    - Wherever you move the blue patch it looks the same.
- The **black patch** has an **edge**. 
    - If you move it in vertical direction (i.e. along the gradient) it **changes**. 
    - Moved along the edge (parallel to edge), it **looks the same**.
- And for **red patch**, it is a **corner**. 
    - Wherever you move the patch, it looks different, means it is unique. 
    - So basically, corners are considered to be good features in an image. <br><br>
- Once you have found it, you should be able to find the same in the other images. 
- How is this done? We take a region around the feature, we explain it in our own words, like "upper part is blue sky, lower part is region from a building, on that building there is glass etc" and you search for the same area in the other images. 
- Basically, you are `describing the feature`. 
Similarly, a computer also should describe the region around the feature so that it can find it in other images. So called description is called **Feature Description**. 
- So in this module, we are looking to different algorithms in OpenCV to **find features**, **describe them**, **match them etc**. <br><br>
![](resources/feature-tech.png)
<br><br>
____
## 1.1 Harris Corner Detection (Edge & Corner Detector)
- **Corners** are regions in the image with **large variation in intensity** in all the directions. 
- One early attempt to find these corners was done by `Chris Harris` & `Mike Stephens` in their paper `A Combined Corner and Edge Detector` in `1988`,
- It basically **finds the difference in intensity** for a displacement of small area with size $(u,v)$ in all directions.
![](resources/harris.png)
- The window function is either a rectangular window or a Gaussian window which gives weights to pixels underneath.
- To detect Corner, we need to find maximum $E(u,v)$.
- Then applying soring function $R$, <br>
    ![](resources/harris_scorring.png)<br><br>
    ![](resources/harris_diagram.png)<br><br>
- Local maxima in 2D image ilustration (vertical value is image depth 0-255)<br>
<img src="resources/local_maxima.png" style="width:400px"></img><br><br>
- OpenCV has the function `cv2.cornerHarris(img, blockSize, ksize, k)` for this purpose. 
- Where : 
    - `img` - Input image. It should be grayscale and float32 type.
    - `blockSize` - It is the size of neighbourhood considered for corner detection
    - `ksize` - Aperture parameter of the Sobel derivative used.
    - `k` - Harris detector free parameter in the equation.<br><br>

In [ ]:
# load image
img = cv2.imread('chessboard.png')

# convert to grayscale with type CV_32FU1
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray32F = gray.astype(np.float32)

# apply Corner Haris with blockSize=2, ksize=3, k=0.04, 
# the corner area has high intensity value compare to other area.
dst = cv2.cornerHarris(gray32F, blockSize=2, ksize=3, k=0.04)


# build mask image to store local maxima coordinate
mask = np.zeros_like(gray)
mask[dst>0.05*dst.max()] = 255

# find contour from detected image corner 
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw cicle on detected contour
for cnt in contours :
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.circle(img, (x + w//2, y + h//2), int(0.02*img.shape[0]), (0, 255, 0), 2)

# show result
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Detected Corner")

plt.subplot(1,2,2)
plt.imshow(mask, cmap="gray")
plt.title("Corner Haris Image after Dillating (type CV_32FU1)")

plt.show()

____
## 1.2 Shi-Tomasi Corner Detector
- Tomasi made a small modification to it in their paper `Good Features to Track` which shows better results compared to `Harris Corner Detector`. 
- The scoring function in Harris Corner Detector was given by:<br>
![](resources/harris_scorring.png)<br><br>
- Instead of this, Shi-Tomasi proposed: <br>
![](resources/shi_scorring_.png)<br><br>
- OpenCV has a function, `cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance, corners, mask, blockSize, useHarrisDetector, k)`. 
- It finds N strongest corners in the image by `Shi-Tomasi method` (or `Harris Corner Detection`, if you specify it). 
- Where : 
    - `image`: Input 8-bit or floating-point 32-bit, single-channel image.
    - `maxCorners`: Maximum number of corners to return. If negative, means nolimit.
    - `qualityLevel`: Parameter characterizing the minimal accepted quality of image corners. 
    - `minDistance`: Minimum possible Euclidean distance between the returned corners.
    - `mask` : Optional region of interest.
    - `blockSize` : It is the size of neighbourhood considered for corner detection
    - `useHarrisDetector` : Parameter indicating whether to use a Harris detector.
    - `k` : Harris detector free parameter in the equation.

In [ ]:
# load image
img = cv2.imread('blox.jpg')

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# detect corner using Shi-Tomasi Detector
corners = cv2.goodFeaturesToTrack(gray, maxCorners=25, qualityLevel=0.01, minDistance=10)

# convert to int 64
corners = corners.astype(np.int0)

# draw circel for all detected corner
for x, y in corners[:,0]:
    cv2.circle(img, (x,y), int(0.02*img.shape[0]), (0, 0, 255), 2)

# show result
plt.figure(figsize=(14,7))
plt.imshow(img[:,:,::-1])

____
## 1.3 SIFT (Scale-Invariant Feature Transform)
- In last couple of chapters, we saw some corner detectors like Harris etc. 
- They are **rotation-invariant**, which means, even if the image is rotated, we can find the same corners. 
- It is obvious because corners remain corners in rotated image also. 
- But what about scaling? A corner may not be a corner if the image is scaled. <br>     
![](resources/sift_scale_invariant.jpg)<br><br>
- SIFT Algorithm :
    - 1. Scale-space Extrema Detection
    - 2. Keypoint Localization   
    - 3. Orientation Assignment
    - 4. Keypoint Descriptor
    - 5. Keypoint Matching<br><br>
- In OpenCV we can use class `cv2.SIFT_create()` to create SIFT Object.
- Then call method `.create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)` to setup parameter (if needed).
- Where : 
    - `nfeatures` : The number of best features to retain. default is 0.
    - `nOctaveLayers` : The number of layers in each octave. default is 3.
    - `contrastThreshold` : The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. default is 0.04.
    - `edgeThreshold`: The threshold used to filter out edge-like features. default is 10. 
    - `sigma` : The sigma of the Gaussian applied to the input image.  default is 1.6.
- Then call `.detect(img, mask)` to detect **keypoint** on image using SIFT.
- Where : 
    - `img` : input image
    - `mask` : optional mask image 
- Then call `.compute(img, keypoint)` to computes the **descriptors** from the keypoints we have found.
- Where : 
    - `img` : input image
    - `keypoint` : detected keypoint from `.detect(img, mask)`
- Optionaly, we cann call `.detectAndCompute(img, mask)` to detect keypoint and compute descrptor at one command.
- Also we can use `cv2.drawKeypoints(gray,kp,img, (255,0,0), 4)` to draw keypoint on image.




In [ ]:
# load image
img = cv2.imread('chessboard.png')

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# create SHIFT object
sift = cv2.SIFT_create()

# detect corner in image using SHIFT
kp = sift.detect(gray,None)

# dray keypoints (detected corners by SIFT)
img = cv2.drawKeypoints(gray, kp, img, (255,0,0), 4)

# compute descriptor 
# descriptor is a numpy array of shape (Number of Keypoints)×128.
kp, des = sift.compute(gray, kp)
print("descriptor shape ", des.shape)

# show result
plt.figure(figsize=(14,7))
plt.imshow(img[:,:,::-1])

plt.show()

____
## 1.4 SURF (Speeded-Up Robust Features)
- we saw SIFT for keypoint detection and description. 
- But it was comparatively **slow** and people needed more speeded-up version. 
<br><br>
- In OpenCV we can use class `cv2.xfeatures2d.SURF_create(hessianThreshold, nOctaves, nOctaveLayers, extended, upright)`
- Where : 
    - `hessianThreshold` : Threshold for hessian keypoint detector used in SURF.
    - `nOctaves` : Number of pyramid octaves the keypoint detector will use.
    - `nOctaveLayers` : Number of octave layers within each octave.
    - `extended` : Extended descriptor flag (true - use extended 128-element descriptors;
    - `upright` : Up-right or rotated features flag (true - do not compute orientation of features;
- Then use method `.detectAndCompute(img, mask)` do detect keypoint and compute descriptor

In [ ]:
img = cv2.imread('butterfly.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Create SURF object. You can specify params here or later.
# Here I set Hessian Threshold to 400
surf = cv2.xfeatures2d.SURF_create(400)

# Check upright flag, if it False, set it to True
surf.setUpright(True)


# Find keypoints and descriptors directly
kp, des = surf.detectAndCompute(gray, None)

print("number of keypoint :", len(kp))

- `1335` keypoints is too much to show in a picture. 
- We reduce it to some `50` to draw it on an image. 
- While matching, we may need all those features, but not now. 
- So we increase the `Hessian Threshold`.

In [ ]:
# Check present Hessian threshold using method `.getHessianThreshold()`
print("Hessian threshold :", surf.getHessianThreshold())

In [ ]:
# We set it to some 50000. Remember, it is just for representing in picture.
# In actual cases, it is better to have a value 300-500
surf.setHessianThreshold(50000)

# Again compute keypoints and check its number.
kp, des = surf.detectAndCompute(gray, None)
print("number of keypoint :", len(kp))

- It is less than 50. Let's draw it on the image.

In [ ]:
img = cv2.drawKeypoints(img, kp, None, (255,0,0), 4)

#show result
plt.figure(figsize=(14,7))
plt.imshow(img[:,:,::-1])

___
## 1.5 FAST (Features from Accelerated Segment Test) Algorithm for Corner Detection
- We saw several feature detectors and many of them are really good. 
- But when looking from a real-time application point of view, they are not fast enough. 
- In OpenCV we can use `cv2.FastFeatureDetector_create(threshold, nonmaxSuppression, type)`
- Where : 
    - `threshold` : default 10.
    - `nonmaxSuppression` : default `true`.
    - type : 
        - `cv2.TYPE_5_8`
        - `cv2.TYPE_7_12 `
        - `cv2.TYPE_9_16`
        - `cv2.THRESHOLD`
        - `cv2.NONMAX_SUPPRESSION`
        - `cv2.FAST_N`
- Then use method `.detect(img, mask)` to detect keypoint
- Then use method `.compute(img, kp)` to compute descriptor
- Or using `.detectAndCompute(img,mask)`

In [ ]:
img = cv2.imread('blox.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Initiate FAST object with default values
fast = cv2.FastFeatureDetector_create(threshold=40)

# find and draw the keypoints
kp = fast.detect(gray, None)
img2 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))

#show result
plt.figure(figsize=(14,7))
plt.imshow(img2[:,:,::-1])
plt.title("FAST default")

In [ ]:
# Disable nonmaxSuppression using method `.setNonmaxSuppression(0)`
fast.setNonmaxSuppression(0)

# re-detect keypoint using FAST
kp = fast.detect(gray, None)

# draw keypoint
img3 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))

#show result
plt.figure(figsize=(14,7))
plt.imshow(img3[:,:,::-1])
plt.title("FAST disable nonmaxSuppression")

# 2. Feature Matching
## 2.1 Basics of Brute-Force Matcher (BFMatcher)
- Brute-Force matcher is simple. 
- It takes the descriptor of one feature in first set and is matched with all other features in second set using some distance calculation. 
- And the closest one is returned using KNN algorithm.

In [ ]:
# EXAMPLE BF MATCHER WITH SURF

img1 = cv2.imread('box.png')
img2 = cv2.imread('box_in_scene.png')

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

surf = cv2.xfeatures2d.SURF_create(400)
surf.setUpright(True)
surf.setHessianThreshold(7000)

# Find keypoints and descriptors directly
kp1, des1 = surf.detectAndCompute(gray1, None)
kp2, des2 = surf.detectAndCompute(gray2, None)

print("number of keypoint 1:", len(kp1))
print("number of keypoint 2:", len(kp2))

In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Apply ratio test
good = []
for m, n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])

# cv.drawMatchesKnn expects list of lists as matches.
result = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

#show result
plt.figure(figsize=(14,7))
plt.imshow(result)
plt.show()

In [ ]:
# EXAMPLE BF MATCHER WITH SIFT

img1 = cv2.imread('box.png')
img2 = cv2.imread('box_in_scene.png')

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(gray1,None)
kp2, des2 = sift.detectAndCompute(gray2,None)

print("number of keypoint 1:", len(kp1))
print("number of keypoint 2:", len(kp2))

In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Apply ratio test
good = []
for m, n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])

# cv.drawMatchesKnn expects list of lists as matches.
result = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

#show result
plt.figure(figsize=(14,7))
plt.imshow(result)
plt.show()

## 2.2 FLANN Matcher (Fast Library for Approximate Nearest Neighbors)
- It contains a collection of algorithms optimized for fast nearest neighbor search in large datasets and for high dimensional features. 
- It works faster than BFMatcher for large datasets. 

In [ ]:
img1 = cv2.imread('box.png')          # queryImage
img2 = cv2.imread('box_in_scene.png') # trainImage

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(gray1,None)
kp2, des2 = sift.detectAndCompute(gray2,None)


# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)


# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]

draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
result = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None,**draw_params)

#show result
plt.figure(figsize=(14,7))
plt.imshow(result)
plt.show()

# Source 
- [https://docs.opencv.org/4.5.3/df/d54/tutorial_py_features_meaning.html](https://docs.opencv.org/4.5.3/df/d54/tutorial_py_features_meaning.html)
- [https://docs.opencv.org/4.5.3/dc/d0d/tutorial_py_features_harris.html](https://docs.opencv.org/4.5.3/dc/d0d/tutorial_py_features_harris.html)
- [https://docs.opencv.org/4.5.3/d4/d8c/tutorial_py_shi_tomasi.html](https://docs.opencv.org/4.5.3/d4/d8c/tutorial_py_shi_tomasi.html)
- [https://docs.opencv.org/4.5.3/da/df5/tutorial_py_sift_intro.html](https://docs.opencv.org/4.5.3/da/df5/tutorial_py_sift_intro.html)
- [https://docs.opencv.org/4.5.3/df/dd2/tutorial_py_surf_intro.html](https://docs.opencv.org/4.5.3/df/dd2/tutorial_py_surf_intro.html)